In [ ]:
import os
import zipfile
import urllib.request

def download_and_extract_zip(url, dest_folder="dataset"):
    zip_filename = url.split("/")[-1]

    zip_path = os.path.join(os.getcwd(), zip_filename)

    print(f"Baixando o arquivo de {url}...")
    urllib.request.urlretrieve(url, zip_path)
    print(f"Arquivo baixado para {zip_path}")

    if not os.path.exists(dest_folder):
        os.makedirs(dest_folder)
        print(f"Pasta '{dest_folder}' criada.")

    print(f"Extraindo o arquivo principal para a pasta '{dest_folder}'...")
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(dest_folder)
    print(f"Arquivo principal extraído com sucesso!")

    os.remove(zip_path)
    print(f"Arquivo ZIP principal removido após extração.")

    nested_zip_path = os.path.join(dest_folder, 'UCI HAR Dataset.zip')

    if os.path.exists(nested_zip_path):
        print(f"Arquivo ZIP encontrado dentro da pasta '{dest_folder}': {nested_zip_path}")
        
        print(f"Extraindo o arquivo ZIP dentro da pasta '{dest_folder}'...")
        with zipfile.ZipFile(nested_zip_path, 'r') as nested_zip_ref:
            nested_zip_ref.extractall(dest_folder)
        print(f"Arquivo extraído com sucesso!")

        os.remove(nested_zip_path)
        print(f"Arquivo ZIP interno removido após extração.")
    else:
        print(f"Nenhum arquivo 'UCI HAR Dataset.zip' encontrado dentro de '{dest_folder}'.")


url = "https://archive.ics.uci.edu/static/public/240/human+activity+recognition+using+smartphones.zip"
download_and_extract_zip(url)

In [ ]:
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

### dataset/UCI HAR Dataset.names

O dataset contém 561 features por amostra, extraídas de dados de aceleração e giroscópio.
Cada amostra possui um rótulo de atividade associado.
Os dados já estão divididos em train e test.

In [ ]:
# Carregar as features
features = pd.read_csv('dataset/UCI HAR Dataset/features.txt', sep=r'\s+', header=None)

# Carregar os dados de treinamento
X_train = pd.read_csv('dataset/UCI HAR Dataset/train/X_train.txt', sep=r'\s+', header=None)
y_train = pd.read_csv('dataset/UCI HAR Dataset/train/y_train.txt', sep=r'\s+', header=None)

# Carregar os dados de teste
X_test = pd.read_csv('dataset/UCI HAR Dataset/test/X_test.txt', sep=r'\s+', header=None)
y_test = pd.read_csv('dataset/UCI HAR Dataset/test/y_test.txt', sep=r'\s+', header=None)

Escale os dados para normalizar as magnitudes:

In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

Reduzindo a dimensionalidade usando PCA (para facilitar a visualização)

In [ ]:
from sklearn.decomposition import PCA

pca = PCA(n_components=2)  # Reduza para 2 dimensões
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(X_train_pca[:, 0], X_train_pca[:, 1], s=5, alpha=0.7, cmap='viridis')
plt.title('Dados Originais em 2D (PCA)')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.show()

In [ ]:
inertia = []
k_values = range(1, 11)

for k in k_values:
    kmeans = KMeans(n_clusters=k, random_state=42)
    kmeans.fit(X_train_scaled)
    inertia.append(kmeans.inertia_)

plt.figure(figsize=(10, 6))
plt.plot(k_values, inertia, marker='o')
plt.title('Método do Cotovelo')
plt.xlabel('Número de Clusters (k)')
plt.ylabel('Inércia')
plt.xticks(k_values)
plt.grid()
plt.show()


Escolhendo o número de clusters (6 para as 6 atividades):

In [ ]:
kmeans = KMeans(n_clusters=6, random_state=42)
kmeans.fit(X_train_scaled)

In [ ]:
clusters = kmeans.predict(X_train_scaled)

In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(X_train_pca[:, 0], X_train_pca[:, 1], c=clusters, cmap='viridis', s=5)
plt.title('Clusters Identificados pelo K-Means')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.colorbar(label='Cluster')
plt.show()


In [ ]:
plt.figure(figsize=(10, 6))
plt.scatter(X_train_pca[:, 0], X_train_pca[:, 1], c=y_train.values.ravel(), cmap='tab10', s=5)
plt.title('Agrupamento Real (Baseado nos Rótulos)')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.colorbar(label='Atividade')
plt.show()

In [ ]:
plt.scatter(X_train_pca[:, 0], X_train_pca[:, 1], c=clusters, cmap='viridis', s=5)
plt.title('Clusters gerados pelo K-Means')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=6, random_state=42, max_iter=300, n_init=10, verbose=1)
kmeans.fit(X_train_scaled)

centers = kmeans.cluster_centers_
if X_train_scaled.shape[1] > 2:
    centers_pca = pca.transform(centers)
else:
    centers_pca = centers

plt.figure(figsize=(10, 6))
plt.scatter(X_train_pca[:, 0], X_train_pca[:, 1], c=clusters, cmap='viridis', s=5, alpha=0.7)
plt.scatter(centers_pca[:, 0], centers_pca[:, 1], c='red', marker='X', s=200, label='Centros')
plt.title('Centros dos Clusters')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.legend()
plt.show()


In [ ]:
from sklearn.metrics import adjusted_rand_score

ari = adjusted_rand_score(y_train.values.ravel(), clusters)
print(f'Adjusted Rand Index: {ari}')

In [ ]:
from sklearn.metrics import silhouette_score

score = silhouette_score(X_train_scaled, clusters)
print(f'Silhouette Score: {score}')